# 使用Torch来构建RNN网络

在Torch中可以采用nngraph组件来构建RNN，通过nngraph可以构造复杂结构的多输入、多输出的有向网络。

In [15]:
require('nngraph')

true	


## 构造多输入到单一输出

这里构造一个两个向量输入，一个向量输出的简单网络，如下所示：

<img src="./images/nto1.jpg">

In [16]:
-- 构造神经网络每个独立的组件
linearLayer1 = nn.Linear(3,2)
linearLayer2 = nn.Linear(2,2)
addLayer = nn.CAddTable()
tanhLayer = nn.Tanh()
linearLayer3 = nn.Linear(2,1)

In [21]:
-- 构造多输入到单一输出的网络
local inNode1 = linearLayer1()    -- 空括号表示，输入由runtime决定
local inNode2 = linearLayer2()
local addNode = addLayer({inNode1,inNode2})  -- ()表示输入Node
local tanhNode = tanhLayer(addNode)
local outNode = linearLayer3(tanhNode)

model = nn.gModule({inNode1, inNode2}, {outNode})


In [23]:
x1 = torch.Tensor({0.1, 1.5, -1.0})
x2 = torch.Tensor({-1, 0})
local y = model:forward({x1,x2})
print(y)

-0.1156
[torch.DoubleTensor of size 1]



In [24]:
-- 手动检验一下
local l1 = linearLayer1:forward(x1)
local l2 = linearLayer2:forward(x2)
local add = addLayer:forward({l1,l2})
local yp = tanhLayer:forward(add)
local y = linearLayer3:forward(yp)
print(y)

-0.1156
[torch.DoubleTensor of size 1]



##  多输入到多输出
和之前的例子一样，只是增加一个输出，构造的网路如下所示：

<img src="./images/nton.jpg">


In [25]:
-- 构造多输入到多输出的网络
local inNode1 = linearLayer1()    -- 空括号表示，输入由runtime决定
local inNode2 = linearLayer2()
local addNode = addLayer({inNode1,inNode2})  -- ()表示输入Node
local tanhNode = tanhLayer(addNode)
local outNode = linearLayer3(tanhNode)

model = nn.gModule({inNode1, inNode2}, {outNode, addNode})    -- 输出增加一项

In [26]:
y = model:forward({x1,x2})
print(y)

{
  1 : DoubleTensor - size: 1
  2 : DoubleTensor - size: 2
}


In [28]:
print(y[1], y[2])

-0.1156
[torch.DoubleTensor of size 1]

-0.3330
-1.1955
[torch.DoubleTensor of size 2]



##  构造递归结构

由于我们得到了中间变量的输出，因此可以把中间变量给输出到下一次的输入中。


In [32]:
local h0 = torch.rand(2)
local x_t0 = torch.rand(3)
local out_t1 = model:forward({x_t0, h0})

print(out_t1[1])

local x_t1 = torch.rand(3)
local h1 = out_t1[2]                -- 得到h1项
local out_t2 = model:forward({x_t1, h1}) 

print(out_t2[1])


-0.6251
[torch.DoubleTensor of size 1]

-0.5163
[torch.DoubleTensor of size 1]

